In [ ]:
# default_exp core

In [ ]:
!which python

/home/phs/miniconda3/envs/yields_release/bin/python


# Core (deprecated)

> Adapt the simpletransformer ClassificationModel to use a SmilesTokenizer (deprecated). The models can now direclty be imported from the `rxnfp` package

In [ ]:
#export
import os
import numpy as np
import torch
import random
import pkg_resources

from rxnfp.models import SmilesClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
This extension has only been tested with simpletransformers==0.34.4


### Load model

Here we use the pretrained reaction BERT model from the `rxnfp` module as test.

In [ ]:
model_path =  pkg_resources.resource_filename(
                "rxnfp",
                f"models/transformers/bert_pretrained"
)
yield_bert = SmilesClassificationModel('bert', model_path, use_cuda=False)

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [ ]:
assert yield_bert.tokenizer.tokenize('c1ccccc1') == ['c', '1', 'c', 'c', 'c', 'c', 'c', '1']